In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from numba import njit
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import random


In [ ]:
# Parameters
T = 10000 # iterations 
k = 6 # price interval 1/k
alpha = 0.3 # learning rate
delta = 0.95 # discount factor
theta = (-(1/T) ** i + 1 for i in range(T))
epsilon = ((1 - theta) ** i for i in range(T))
# initialising q-tables
q1 = np.zeros((k, k))
q2 = np.zeros((k, k))
p_table = np.linspace(1/k, 1, k)
print(np.array([epsilon]))


[<generator object <genexpr> at 0x7fca98ac33c0>]


In [ ]:
import numpy as np

# Parameters
T = 10000  # iterations
k = 6  # price interval 1/k
alpha = 0.3  # learning rate
delta = 0.95  # discount factor

# Creating sequences for theta and epsilon
theta = np.array([(-(1 / T) ** i + 1) for i in range(T)])
epsilon = np.array([(1 - theta_i) ** i for i, theta_i in enumerate(theta)])

# Initializing q-tables
q1 = np.zeros((k, k))
q2 = np.zeros((k, k))
p_table = np.linspace(1 / k, 1, k)

print(epsilon)


[1.00000000e+00 1.00000000e-04 1.00000001e-16 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [ ]:
def demand(p1t,p2t):
    """
    args:
        p1t: price of agent 1
        p2t: price of agent 2
    returns:
        d: demand for given set of prices
    """
    if p1t<p2t:
        d = 1-p1t
    elif p1t==p2t:
        d = 0.5*(1-p1t)
    else:
        d = 0
    return d

<>:12: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:12: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
/var/folders/gr/gsfm7155227gh8vpmf4v3g840000gn/T/ipykernel_67596/3186152009.py:12: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  d = 0.5(1-p1t)


In [4]:
demand(0.1667,0.5)

NameError: name 'demand' is not defined

In [9]:
def profit(p1t, p2t):
    """
    args:
        p1t: index price of agent 1
        p2t: index price of agent 2
    returns:
        profit for agent
    """
    return p_table[p1t] * demand(p_table[p1t], p_table[p2t]) 

In [10]:
np.random.choice(len(p_table))


2

In [11]:
def select_price_greedy(Q, current_s, p, epsilon):
    """Epsilon-greedy action selection.
    args:
        Q: Q-function
        current_s: current state
        p: price vector containing the possible prices
        epsilon: probability of selecting an action uniformly at random
        u: draw from U(0,1) distribution
    returns:
        the index of selected action
    """
    u = random.uniform(0,1)
    if u < epsilon:
        #return np.random.choice(p)
        random_index = np.random.choice(len(p_table))
        return random_index
    else:
        print(Q)
        max_idx = np.argmax(Q[np.where(p == current_s)[0][0], :])
        #max_idx = np.argmax(Q[current_s, :])
        return max_idx

## Q-function

In [22]:
def Q_func(p_it, s_t, q_table,iteration):
    """
    args
        p_it: current price index
        s_t: current state index
        q_table: q_table for 1 player
    returns:
        updated Q-table
    """
    prev_est = q_table[p_it,s_t]
    s_t1 = select_price_greedy(q_table, p_table[s_t], p_table, epsilon[iteration])
    print(s_t1)
    new_est = profit(p_it, s_t) + delta * profit(p_it, s_t1) + delta**2 * np.argmax(q_table[p_it, s_t1])
    q_table[p_it,s_t] = (1 - alpha) * prev_est + alpha * new_est
    return q_table


In [23]:
print(Q_func(2, 2, q1,2))

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
0


TypeError: 'float' object is not callable

In [24]:
demand(0.1667,0.50)

0.8333

In [25]:
print(p_table)

[0.16666667 0.33333333 0.5        0.66666667 0.83333333 1.        ]


In [26]:
def simulation(num_sim):
    SavedQFunc1=[]
    SavedQFunc2=[]
    for i in range(num_sim): 
        q1 = np.zeros((k, k))
        q2 = np.zeros((k, k))
        c_state = 1
        print("abe")
        for j in range(1,T):
            print("hval")
            if j%2 == 1:
                p1 = select_price_greedy(q1, c_state, p_table, epsilon[j])
                Q_func(p1, c_state, q1, j)
                c_state = p1
                print(c_state)
            else:
                print("abdi")
                p2 = select_price_greedy(q2, c_state, p_table, epsilon[j])
                Q_func(p2, c_state, q2, j)
                c_state = p2
        print("antilope")
        SavedQFunc1.append(q1)
        SavedQFunc2.append(q2)
    print("ko")
    return (SavedQFunc1,SavedQFunc2)


In [29]:
def Q_func(p_it, s_t, q_table, iteration):
    """
    args
        p_it: current price index
        s_t: current state index
        q_table: q_table for 1 player
    returns:
        updated Q-table
    """
    prev_est = q_table[p_it, s_t]
    s_t1 = select_price_greedy(q_table, s_t, p_table, epsilon[iteration])
    reward = profit(p_it, s_t)
    next_max_q = np.max(q_table[p_it, s_t1])
    new_est = (1 - alpha) * prev_est + alpha * (reward + delta * next_max_q)
    q_table[p_it, s_t] = new_est
    return q_table


In [ ]:

r1,r2=simulation(2)



Simulation: 0
Iteration: 1
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Agent 1 selected price: 0.16666666666666666
Iteration: 2
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


IndexError: index 0 is out of bounds for axis 0 with size 0